# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

In [1]:
import xlwings as xw
import numpy as np
import pandas as pd

In [2]:
xw.__version__

'0.24.3'

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [3]:
wb = xw.Book('себестоимостьА_в1.xlsx')

In [4]:
sheet = wb.sheets['Рецептура']
consumption = sheet.range('G7:O10').options(np.array).value
consumption

array([[0.2  , 0.4  , 0.02 , 0.02 , 0.01 , 0.005, 0.1  ,   nan,   nan],
       [0.07 , 0.25 , 0.01 , 0.06 , 0.015,   nan, 0.06 , 0.05 ,   nan],
       [0.12 , 0.22 , 0.005,   nan, 0.009, 0.005,   nan, 0.05 , 0.03 ],
       [0.12 , 0.23 , 0.01 ,   nan, 0.015,   nan,   nan,   nan, 0.05 ]])

In [5]:
unit_price = sheet.range('G14:O14').options(np.array).value
unit_price

array([  7.,  10., 184.,  19.,   7., 290., 105.,  83., 260.])

In [6]:
np.nan_to_num(consumption * unit_price).sum(axis = 1)

array([21.48 , 16.525, 17.423, 18.085])

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [7]:
sheet.range('T7:T10').options(transpose=True).value = np.nan_to_num(consumption * unit_price).sum(axis = 1)

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [8]:
sheet.range('T6').value = 'Себестоимость'

In [9]:
sheet.range('T4:T6').api.merge()

4. Выполнить 3 с помощью "протягиваемых" формул.

In [10]:
from xlwings.constants import AutoFillType

In [11]:
sheet.range('V7').formula = '=SUMPRODUCT(G7:O7, $G$14:$O$14)'
sheet.range('V7').api.autofill(destination=sheet.range('V7:V10').api, type=AutoFillType.xlFillDefault)

## Лабораторная работа 7

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [12]:
recipes = pd.read_csv('recipes_sample_with_tags_ingredients.csv')[['id','name','minutes','submitted','description','n_ingredients']]
reviews = pd.read_csv('reviews_sample.csv', index_col = 0)

In [13]:
reviews.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [14]:
app = xw.App()
wb = app.books.add()
wb.sheets.add()
wb.save('recipes.xlsx')
wb.close() 
app.quit()

In [15]:
dz = xw.Book('recipes.xlsx')
dz.sheets[0].name = ['Рецепты']
dz.sheets[1].name = ['Отзывы']

In [16]:
recipes_005 = recipes.sample(frac = 0.05, replace = False, random_state = 333)
reviews_005 = reviews.sample(frac = 0.05, replace = False, random_state = 333)

In [17]:
sheet_1 = dz.sheets[0]
sheet_2 = dz.sheets[1]

In [18]:
sheet_1.range('A1').value = recipes_005
sheet_2.range('A1').value = reviews_005

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [19]:
minutes = sheet_1.range('D2:D1501').options(np.array).value
minutes

array([115.,  40.,  65., ...,  90.,  35.,   5.])

In [20]:
sheet_1.range('H2:H1501').options(transpose=True).value = np.nan_to_num(minutes * 60)
sheet_1.range('H1').value = 'seconds_assign'

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [21]:
sheet_1.range('I1').value = 'seconds_formula'

In [22]:
sheet_1.range('I2').formula = '=D2*60'
sheet_1.range('I2').api.autofill(destination = sheet_1.range('I2:I1501').api, type=AutoFillType.xlFillDefault)

5. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [23]:
sheet_1.range('J1').value = 'n_reviews'

In [24]:
sheet_1.range('J2').formula = '=COUNTIF(Отзывы!$C$2:$C$6336,Рецепты!B2)'

In [25]:
sheet_1.range('J2').api.autofill(destination = sheet_1.range('J2:J1501').api, type=AutoFillType.xlFillDefault)

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [30]:
sheet_1.range('H1:J1').font.bold = True
sheet_1.range('H1:J1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

7. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [27]:
for x in sheet_1.range('D2:D1501'):
    if x.value < 5:
        x.color = (0,128,0)
    elif 5 <= x.value <= 10:
        x.color = (255,255,0)
    elif x.value > 10:
        x.color = (255,0,0)

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [28]:
def validate(wb):
    rec = wb.sheets['Рецепты']
    rev = wb.sheets['Отзывы']
    id_rec = np.array(rec.range('B2:B1501').value)
    for i in range(2, 6336):
        if((0 <= rev.range(f'E{i}').value <= 5) and (rev.range(f'C{i}').value in id_rec)):
            continue
        else:
            rev.range(f'B{i}:F{i}').color = (255,0,0)
    return

In [29]:
validate(dz)